# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

This is an investing strategy that selects the 50 stocks with the highest price momentum.


## Library Imports

In [52]:
import numpy as np 
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter

## Importing Our List of Stocks

In [53]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## API Call

We need to get one-year price returns for each stock in the universe. 

In [54]:
symbol = 'AAPL'
api_url =  f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data.

SyntaxError: invalid syntax (<ipython-input-54-2e1e93aa8116>, line 4)

## Parsing Our API Call

In [55]:
data['year1ChangePercent']

KeyError: 'year1ChangePercent'

## Executing A Batch API Call & Building Our DataFrame

In [56]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

for symbol_string in symbol_strings:
    print(symbol_string)

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA
CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD
GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILM

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [57]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,121.650,0.418962,N/A
1,AAL,16.962,-0.383379,N/A
2,AAP,163.270,0.0377098,N/A
3,AAPL,131.693,0.882249,N/A
4,ABBV,106.360,0.244216,N/A
...,...,...,...,...
500,YUM,108.550,0.0943888,N/A
501,ZBH,153.470,0.00625074,N/A
502,ZBRA,395.720,0.504056,N/A
503,ZION,42.070,-0.161683,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.


In [58]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,CARR,38.000,2.18561,N/A
1,AMD,97.880,1.42637,N/A
2,NVDA,537.510,1.40607,N/A
3,LB,40.200,1.27329,N/A
4,ALB,138.400,1.12724,N/A
5,PYPL,232.000,1.06071,N/A
6,ALGN,535.610,0.947925,N/A
7,NOW,571.000,0.932447,N/A
8,FCX,24.580,0.903539,N/A
9,AAPL,131.693,0.882249,N/A


## Calculating the Number of Shares to Buy

In [72]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of your portfolio:')
    
    try:
        float(portfolio_size)
    except ValueError:
        print('That is not a number, please try again:')
        portfolio_size = input('Enter the size of your portfolio:') 
        
portfolio_input()
print(portfolio_size)

Enter the size of your portfolio:10000000
10000000


In [79]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe.loc[i, 'Price'])

final_dataframe

/Users/caseybrugna/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,CARR,38.000,2.18561,5159
1,AMD,97.880,1.42637,2003
2,NVDA,537.510,1.40607,364
3,LB,40.200,1.27329,4877
4,ALB,138.400,1.12724,1416
5,PYPL,232.000,1.06071,845
6,ALGN,535.610,0.947925,366
7,NOW,571.000,0.932447,343
8,FCX,24.580,0.903539,7977
9,AAPL,131.693,0.882249,1488


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns


In [87]:

hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series(
                                            [
                                                symbol, 
                                                data[symbol]['quote']['latestPrice'],
                                                'N/A',
                                                data[symbol]['stats']['year1ChangePercent'],
                                                'N/A',
                                                data[symbol]['stats']['month6ChangePercent'],
                                                'N/A',
                                                data[symbol]['stats']['month3ChangePercent'],
                                                'N/A',
                                                data[symbol]['stats']['month1ChangePercent'],
                                                'N/A',
                                                'N/A'
                                            ], 
                                            index = hqm_columns), 
                                            ignore_index = True
                                        )
        
hqm_dataframe.columns

Index(['Ticker', 'Price', 'Number of Shares to Buy', 'One-Year Price Return',
       'One-Year Return Percentile', 'Six-Month Price Return',
       'Six-Month Return Percentile', 'Three-Month Price Return',
       'Three-Month Return Percentile', 'One-Month Price Return',
       'One-Month Return Percentile', 'HQM Score'],
      dtype='object')

## Calculating Momentum Percentiles

In [94]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        if hqm_dataframe.loc[row, f'{time_period} Price Return'] == None:
            hqm_dataframe.loc[row, f'{time_period} Price Return'] = 0

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,118.230,N/A,0.427378,0,0.369702,0,0.180094,0,0.0685384,0,N/A
1,AAL,17.055,N/A,-0.39354,0,0.0190482,0,0.307634,0,0.389996,0,N/A
2,AAP,166.810,N/A,0.0368398,0,0.169466,0,0.0205598,0,0.0386547,0,N/A
3,AAPL,133.621,N/A,0.881516,0,0.504625,0,0.113747,0,0.0748616,0,N/A
4,ABBV,109.040,N/A,0.246175,0,0.134658,0,0.149231,0,0.0392973,0,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,108.260,N/A,0.0945087,0,0.188821,0,0.155237,0,0.0507061,0,N/A
501,ZBH,152.420,N/A,0.00622763,0,0.172905,0,0.0271969,0,-0.0242286,0,N/A
502,ZBRA,383.020,N/A,0.510415,0,0.4959,0,0.497273,0,0.10971,0,N/A
503,ZION,41.910,N/A,-0.162339,0,0.179751,0,0.350615,0,0.113797,0,N/A


## Calculating the HQM Score

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

In [95]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [96]:
hqm_dataframe.sort_values(by = 'HQM Score', ascending = False)
hqm_dataframe = hqm_dataframe[:51]

## Calculating the Number of Shares to Buy


In [97]:
portfolio_input()

Enter the size of your portfolio:100000


In [98]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])-1):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe

/Users/caseybrugna/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,118.230,16,0.427378,0,0.369702,0,0.180094,0,0.0685384,0,0
1,AAL,17.055,114,-0.39354,0,0.0190482,0,0.307634,0,0.389996,0,0
2,AAP,166.810,11,0.0368398,0,0.169466,0,0.0205598,0,0.0386547,0,0
3,AAPL,133.621,14,0.881516,0,0.504625,0,0.113747,0,0.0748616,0,0
4,ABBV,109.040,17,0.246175,0,0.134658,0,0.149231,0,0.0392973,0,0
5,ABC,103.210,18,0.191169,0,0.0383476,0,0.0495838,0,-0.0865013,0,0
6,ABMD,280.790,6,0.521819,0,0.123422,0,-0.0141831,0,0.0121708,0,0
7,ABT,108.380,18,0.276778,0,0.215495,0,0.0115361,0,-0.0468143,0,0
8,ACN,252.130,7,0.210731,0,0.256166,0,0.0329332,0,0.0225055,0,0
9,ADBE,497.000,3,0.530379,0,0.205569,0,-0.0312797,0,0.0290575,0,0


## Formatting Excel Output

In [102]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File

In [103]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [104]:

column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

In [105]:
writer.save()